## Libraries and setup

In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
pd.set_option("display.max_columns", 500)

## Datasets

- `npidata_pfile`: Data File - File contains the FOIA-disclosable NPPES provider data
- `othername_pfile`: Other Name Reference File - File contains additional Other Names associated with Type 2 NPIs
- `pl_pfile`: Practice Location Reference File - File contains all of the non-primary Practice Locations associated with Type 1 and Type 2 NPIs
- `endpoint_pfile`: Endpoint Reference File - File contains all Endpoints associated with Type 1 and Type 2 NPIs

- Taxonomy code to classification crosswalk: `nucc_taxonomy_210.csv`
- Zip code to CBSA crosswalk: `ZIP_CBSA_122020.xlsx`

Read-in the data from file
**TODO: Remove `nrows` to get all the data**

In [3]:
npidata_raw = pd.read_csv("../data/npidata_pfile_20050523-20210207.csv", nrows=1000)

In [4]:
taxonomy_codes = pd.read_csv("../data/nucc_taxonomy_210.csv")

In [5]:
zips = pd.read_excel("../data/ZIP_CBSA_122017.xlsx", converters={'zip': lambda x: str(x)}, engine='openpyxl')

## Subsetting `npidata`

In [6]:
display(npidata_raw.shape)
display(npidata_raw.head())

(1000, 330)

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

Only keep the needed columns

In [7]:
npidata = pd.concat([
    npidata_raw[['NPI']],
    # Entity Type Code: 1 = Provider (doctors, nurses, etc.) / 2 = Facility (Hospitals, Urgent Care, Doctors Offices) 
    npidata_raw[['Entity Type Code']],
    # Entity Name: Either First/Last or Organization or Other Organization Name contained in the following fields:
    npidata_raw.loc[:, 'Provider Organization Name (Legal Business Name)':'Provider Credential Text'],
    # Address: Business Practice Location (not mailing), contained in the following fields:
    npidata_raw.loc[:, 'Provider First Line Business Practice Location Address':'Provider Business Practice Location Address Postal Code'],
], axis=1)

Check the current shape of the dataframe

In [8]:
display(npidata.shape)
display(npidata.head())

(1000, 14)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0
3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0


**We will handle the Primary Taxonomy separately and left-join back to `npidata` later**

## Handling The Primary Taxonomy


A provider can have up to 15 taxonomy codes, but we want the one which has Primary Switch = Y in the associated 'Healthcare Provider Primary Taxonomy Switch*' field. Note that this does not always occur in spot 1.

In [9]:
npi_taxonomy = pd.concat([
    npidata_raw[['NPI']],
    # The provider's taxonomy code, which is contained in one of the 'Healthcare Provider Taxonomy Code*' columns
    npidata_raw[npidata_raw.columns[pd.Series(npidata_raw.columns).str.startswith('Healthcare Provider Taxonomy Code_')]],
    npidata_raw[npidata_raw.columns[pd.Series(npidata_raw.columns).str.startswith('Healthcare Provider Primary Taxonomy Switch_')]]
], axis=1)

In [10]:
display(npi_taxonomy.head())

,NPI,Healthcare Provider Taxonomy Code_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Primary Taxonomy Switch_15
0,1679576722,207X00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,207RC0000X,207RC0000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,251G00000X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,2085R0202X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,174400000X,207RH0003X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Apply `wide_to_long` to get the matrix of taxonomy in a long format and filter for only the primary taxonomies (**warning: This would get rid of NPI's without primary taxonomies. Do a left-join on `npidata` to fix this**)

In [11]:
# Pivot from widet to long format
npi_taxonomy = pd.wide_to_long(
    npi_taxonomy,
    stubnames=['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch'],
    i=['NPI'], 
    j='primary_taxonomy_index',
    sep="_"
)

# Only keep the primary taxonomy
npi_taxonomy = npi_taxonomy[npi_taxonomy['Healthcare Provider Primary Taxonomy Switch'] == 'Y']

# Housekeeping
npi_taxonomy = npi_taxonomy.reset_index()\
    .drop(columns=['primary_taxonomy_index', 'Healthcare Provider Primary Taxonomy Switch'])\
    .rename({ 'Healthcare Provider Taxonomy Code': 'primary_taxonomy' }, axis=1)

In [12]:
display(npi_taxonomy)

,NPI,primary_taxonomy
0,1679576722,207X00000X
1,1497758544,251G00000X
2,1306849450,2085R0202X
3,1023011178,251G00000X
4,1932102084,207RC0000X
...,...,...
932,1699778829,207X00000X
933,1811990047,333600000X
934,1619970845,3336H0001X
935,1164425336,3336H0001X


## Merging `npidata` and `npi_taxonomy`

Finally, merge back `npi_taxonomy` to `npidata`. Account for npis without `primary_taxonomy` by doing a left-join

In [13]:
npidata = npidata.merge(
    npi_taxonomy,
    how='left',
    on='NPI'
)

In [14]:
display(npidata.head())

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,primary_taxonomy
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0,207X00000X
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X


## Renaming Columns

In [15]:
npidata = npidata.reset_index().rename({
    'NPI': 'npi',
    'Entity Type Code': 'entity_type_code',
    'Provider Organization Name (Legal Business Name)': 'provider_org_name',
    'Provider Last Name (Legal Name)': 'provider_last_name',
    'Provider First Name': 'provider_first_name',
    'Provider Middle Name': 'provider_middle_name',
    'Provider Name Prefix Text': 'provider_name_prefix',
    'Provider Name Suffix Text': 'provider_name_suffix',
    'Provider Credential Text': 'provider_credential',
    'Provider First Line Business Practice Location Address': 'provider_business_address_1',
    'Provider Second Line Business Practice Location Address': 'provider_business_address_2',
    'Provider Business Practice Location Address City Name': 'provider_business_city',
    'Provider Business Practice Location Address State Name': 'provider_business_state',
    'Provider Business Practice Location Address Postal Code': 'provider_business_zip'
}, axis=1)

In [16]:
display(npidata.head())

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy
0,0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0,207X00000X
1,1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X
4,4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X


**Note: Not all NPIs has a primary taxonomy**

In [17]:
display(npidata[npidata['primary_taxonomy'].isna()])

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy
35,35,1891798849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,43,1780687731,1.0,NaN,STAUBSINGER,ARLENE,BETH,DR.,NaN,PH.D.,8100 OSWEGO RD,STE 235,LIVERPOOL,NY,130901660.0,NaN
52,52,1770586729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,86,1659374684,2.0,NYDIC OPEN MRI OF AMERICA-BOARDMAN,NaN,NaN,NaN,NaN,NaN,NaN,1449 BOARDMAN CANFIELD RD,STE 140,BOARDMAN,OH,445128070.0,NaN
89,89,1386647311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,956,1538162763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,962,1891798021,1.0,NaN,MORRIS,BRIAN,NaN,NaN,NaN,RPH,1926 DORMINEY CT,NaN,LAWRENCEVILLE,GA,300436913.0,NaN
967,967,1346243573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
973,973,1609879832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add Taxonomy Code to Classification Crosswalk

In [18]:
display(taxonomy_codes.shape)
display(taxonomy_codes.head())

(865, 10)

,Code,Grouping,Classification,Specialization,Definition,Effective Date,Deactivation Date,Last Modified Date,Notes,Display Name
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,10/1/2003,NaN,NaN,[7/1/2003: new],Multi-Specialty Group
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,10/1/2003,NaN,NaN,[7/1/2003: new],Single Specialty Group
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,4/1/2003,NaN,7/1/2007,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,4/1/2003,NaN,NaN,NaN,Allergy Physician
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,4/1/2003,NaN,NaN,NaN,Clinical & Laboratory Immunology (Allergy & Im...


Keep only the columns we need

In [19]:
taxonomy_codes = taxonomy_codes[['Code', 'Classification']]

In [20]:
display(taxonomy_codes.head())

,Code,Classification
0,193200000X,Multi-Specialty
1,193400000X,Single Specialty
2,207K00000X,Allergy & Immunology
3,207KA0200X,Allergy & Immunology
4,207KI0005X,Allergy & Immunology


## Merge `npidata` and `taxonomy`

Merge to `npidata`, matching on primary_taxonomy/Code

In [21]:
npidata = npidata.merge(
    taxonomy_codes, 
    how='left', 
    left_on = 'primary_taxonomy', 
    right_on = 'Code', 
copy=False).drop('Code', axis=1)

In [22]:
display(npidata.head())

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy,Classification
0,0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0,207X00000X,Orthopaedic Surgery
1,1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X,Internal Medicine
2,2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X,"Hospice Care, Community Based"
3,3,1306849450,1.0,NaN,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,NaN,ATHENS,TX,757513446.0,2085R0202X,Radiology
4,4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X,Internal Medicine


## Add Zip Code to CBSA Crosswalk

Match each provider to a CBSA using the Business Zip code. A zipcode can be associated to multiple CBSAs, so match with the CBSA with the highest `TOT_RATIO`

In [23]:
display(zips.head())

,zip,cbsa,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00612,11640,0.996207,0.999588,0.995662,0.996515
1,00627,11640,1.000000,1.000000,1.000000,1.000000
2,00638,41980,1.000000,1.000000,1.000000,1.000000
3,00676,10380,1.000000,1.000000,1.000000,1.000000
4,00719,41980,1.000000,1.000000,1.000000,1.000000


Filter to zips within the Nashville cbsa

In [24]:
zips = zips[zips['cbsa'] == 34980]

Each zip code can be in multiple counties, though we only want to match zips to the county with the greatest representation. To do this, groupby `county`, and keep only the rows with the max `tot_ratio`.

In [25]:
zips = zips.groupby('zip')['tot_ratio'].max().reset_index()

In [26]:
display(zips)

,zip,tot_ratio
0,37010,0.411630
1,37011,1.000000
2,37012,0.216487
3,37013,1.000000
4,37014,1.000000
...,...,...
158,38563,1.000000
159,38567,0.941315
160,38569,0.292111
161,38575,0.001708


Create a list of counties we will use as a filter for the nationwide npidata

In [27]:
zips_list = zips['zip'].to_list()

In [28]:
print(zips_list)

['37010', '37011', '37012', '37013', '37014', '37015', '37016', '37018', '37020', '37022', '37024', '37025', '37026', '37027', '37029', '37030', '37031', '37032', '37033', '37035', '37036', '37037', '37043', '37046', '37048', '37049', '37051', '37052', '37055', '37056', '37057', '37060', '37062', '37063', '37064', '37065', '37066', '37067', '37068', '37069', '37070', '37071', '37072', '37073', '37074', '37075', '37076', '37077', '37080', '37082', '37083', '37085', '37086', '37087', '37088', '37089', '37090', '37091', '37095', '37098', '37101', '37110', '37115', '37116', '37118', '37119', '37121', '37122', '37127', '37128', '37129', '37130', '37131', '37132', '37133', '37135', '37137', '37138', '37140', '37141', '37143', '37145', '37146', '37148', '37149', '37150', '37151', '37152', '37153', '37160', '37165', '37166', '37167', '37171', '37172', '37174', '37179', '37180', '37181', '37184', '37186', '37187', '37188', '37189', '37190', '37201', '37202', '37203', '37204', '37205', '37206', 

Filter npidata to only where zipcodes are within Nashville cbsa

In [29]:
# Start by converting provider_business_zip to string and indexing to first 5 digits
npidata['provider_business_zip5'] = [str(i)[0:5] for i in npidata['provider_business_zip']]

In [30]:
# Filter rows to 'provider_business_zip5' being in zips_list
npidata_nashville = npidata[npidata['provider_business_zip5'].isin(zips_list)]

In [31]:
display(npidata.shape)
display(npidata_nashville.shape)
display(npidata_nashville.head())

(1000, 18)

(30, 18)

,index,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,primary_taxonomy,Classification,provider_business_zip5
48,48,1134122187,1.0,NaN,RUDNICKE,CHERYL,DENISE,MRS.,NaN,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,372031632.0,363L00000X,Nurse Practitioner,37203
67,67,1003819046,1.0,NaN,NYLANDER,BARBARA,H,NaN,NaN,M.D.,345 23RD AVE N,SUITE 209,NASHVILLE,TN,372031513.0,207VG0400X,Obstetrics & Gynecology,37203
76,76,1750384780,1.0,NaN,PERRIGIN,JULIE,A,DR.,NaN,MD,219 CHURCH ST,NaN,DICKSON,TN,370551303.0,207Q00000X,Family Medicine,37055
83,83,1922001957,1.0,NaN,PRESLEY,RICHARD,E,NaN,NaN,M.D.,2011 MURPHY AVE,STE 302,NASHVILLE,TN,372032023.0,207V00000X,Obstetrics & Gynecology,37203
105,105,1073516001,1.0,NaN,ROSS,DAVID,L,DR.,NaN,MD,127 CRESTVIEW PARK DR,NaN,DICKSON,TN,370552850.0,207R00000X,Internal Medicine,37055


## Now re-applying all this at once and import into sqlite

**IMPORTANT! This loading into the database should only be run once.** If you run this multiple times, it will create duplicate entries in the database. For the security of not re-running this code by accident, the code here is converted into markdown. **If you need to rebuild the database, delete the `data/hcbb.sqlite` file and re-run this cell as code. You will also need to make sure to re-run any other related scripts that builds other tables in the database.**

taxonomy_codes = pd.read_csv("../data/nucc_taxonomy_210.csv")
zips = pd.read_excel("../data/ZIP_CBSA_122017.xlsx", converters={'zip': lambda x: str(x)}, engine='openpyxl')
zips = zips[zips['cbsa'] == 34980]

with sqlite3.connect('../data/hcbb.sqlite') as db:

    # Loading the nppes dataset
    npidata_raw = pd.read_csv("../data/npidata_pfile_20050523-20210207.csv", chunksize = 10000)
    
    # Looping over chuncks of nppes
    for chunk in tqdm(npidata_raw):

        npidata = pd.concat([
            chunk[['NPI']],
            # Entity Type Code: 1 = Provider (doctors, nurses, etc.) / 2 = Facility (Hospitals, Urgent Care, Doctors Offices) 
            chunk[['Entity Type Code']],
            # Entity Name: Either First/Last or Organization or Other Organization Name contained in the following fields:
            chunk.loc[:, 'Provider Organization Name (Legal Business Name)':'Provider Credential Text'],
            # Address: Business Practice Location (not mailing), contained in the following fields:
            chunk.loc[:, 'Provider First Line Business Practice Location Address':'Provider Business Practice Location Address Postal Code'],
        ], axis=1)

        npi_taxonomy = pd.concat([
            chunk[['NPI']],
            # The provider's taxonomy code, which is contained in one of the 'Healthcare Provider Taxonomy Code*' columns
            chunk[chunk.columns[pd.Series(chunk.columns).str.startswith('Healthcare Provider Taxonomy Code_')]],
            chunk[chunk.columns[pd.Series(chunk.columns).str.startswith('Healthcare Provider Primary Taxonomy Switch_')]]
        ], axis=1)

        # Pivot from wide to long format
        npi_taxonomy = pd.wide_to_long(
            npi_taxonomy,
            stubnames=['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch'],
            i=['NPI'], 
            j='primary_taxonomy_index',
            sep="_"
        )

        # Only keep the primary taxonomy
        npi_taxonomy = npi_taxonomy[npi_taxonomy['Healthcare Provider Primary Taxonomy Switch'] == 'Y']

        # Housekeeping
        npi_taxonomy = npi_taxonomy.reset_index()\
            .drop(columns=['primary_taxonomy_index', 'Healthcare Provider Primary Taxonomy Switch'])\
            .rename({ 'Healthcare Provider Taxonomy Code': 'taxonomy_code' }, axis=1)

        # Merge npidata with npi_taxonomy
        npidata = npidata.merge(
            npi_taxonomy,
            how='left',
            on='NPI'
        )

        # Rename columns
        npidata = npidata.reset_index().rename({
            'NPI': 'npi',
            'Entity Type Code': 'entity_type_code',
            'Provider Organization Name (Legal Business Name)': 'provider_org_name',
            'Provider Last Name (Legal Name)': 'provider_last_name',
            'Provider First Name': 'provider_first_name',
            'Provider Middle Name': 'provider_middle_name',
            'Provider Name Prefix Text': 'provider_name_prefix',
            'Provider Name Suffix Text': 'provider_name_suffix',
            'Provider Credential Text': 'provider_credential',
            'Provider First Line Business Practice Location Address': 'provider_business_address_1',
            'Provider Second Line Business Practice Location Address': 'provider_business_address_2',
            'Provider Business Practice Location Address City Name': 'provider_business_city',
            'Provider Business Practice Location Address State Name': 'provider_business_state',
            'Provider Business Practice Location Address Postal Code': 'provider_business_zip'
        }, axis=1)

        # Create Zip5 column to merge down the road
        npidata['provider_business_zip5'] = [str(i)[0:5] for i in npidata['provider_business_zip']]

        # Correct data types
        npidata['npi'] = npidata['npi'].astype(str)
        npidata['entity_type_code'] = npidata['entity_type_code'].astype(str).str.split('.').str[0]
        npidata['provider_business_zip'] = npidata['provider_business_zip'].astype(str).str.split('.').str[0]
        npidata['provider_business_zip5'] = npidata['provider_business_zip5'].astype(str).str.split('.').str[0]
        
        # Filter by zips
        npidata = npidata[npidata['provider_business_zip5'].isin(zips['zip'])]
        
        # Remove unneeded columns
        npidata = npidata.drop('index', axis=1)
        
        # Load to sqlite db
        npidata.to_sql('npidata', db, if_exists = 'append', index = False)

    print('task done')

In [7]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT * FROM taxonomy
    LIMIT 5;
    """ 
    
    test_df = pd.read_sql(query, db)

display(test_df)

,taxonomy_code,grouping,classification,specialization
0,193200000X,Group,Multi-Specialty,None
1,193400000X,Group,Single Specialty,None
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology


**Quick Fix for dropping tables (DO NOT RUN UNLESS FOR RECREATING TABLES)**

In [ ]:
# with sqlite3.connect('../data/hcbb.sqlite') as db :
#     cursor = db.cursor()
#     cursor.execute("DROP TABLE npidata")
#     print("Table dropped...")